Attributes of a task:

- Task name
- Number of hours in a week
- Start day and time (optional)
- End day and time (optional)
- is the task Optional
- is the duration of the task flexible (For example, if we can schedule working on a project for 10 hours instead of 12, then that's allowed)
- Duration of continuous work (after which we need a break)
- Min allotted duration of work

Other constraints:
- Taking a break while switching between tasks
- Working window of a person everyday
- Preferred working hours each weekday
- Preferred working hours during weekends

In [1]:
# CONSTANTS and HELPER FUNCTIONS

# This value is used for optional input parameters, in case no fixed value is supplied
default = 0

# Days in the week
num_days = 7
day_names = ["Saturday",
"Sunday",
"Monday",
"Tuesday",
"Wednesday",
"Thursday",
"Friday"]

@enum Days begin
    Saturday = 1
    Sunday = 2
    Monday = 3
    Tuesday = 4
    Wednesday = 5
    Thursday = 6
    Friday = 7
end

# Working hours in a day start at 10:30 (10:30am)
work_start = "10:30"

# Working hours end at 23:30 (11:30pm)
work_end = "23:30"

# We divide the day into 15 minute time slots
num_slots_per_day = Int((24*60)/15)

# Preferred working hours in the weekday and weekends
preferred_working_hours_weekday = 9
preferred_working_hours_weekend = 3

# We divide the day into 15 minute time slots. This function returns the
# time slot index corresponding to a given time in 24-hour HH:MM format.
# Minutes should be a multiple of 15. Time slot index of 00:00 is 1.
function convertTimeToSlotIndex(t)
    typeassert(t, String)
    hour, mins = split(t, ":")
    hour = parse(Int, hour)
    mins = parse(Int, mins)
    if mins%15 != 0
        print("Invalid time input")
    end
    return Int((hour*60 + mins)/15 + 1)
end

function convertSlotIndexToTime(idx)
    typeassert(idx, Int)
    hour = div(idx-1, 4)
    mins = ((idx-1)%4)*15
    time = ""
    if hour < 12
        time = time*"0"*string(hour)
    end
    time = time*":"
    if mins == 0
        time = time*"00"
    else
        time = time*string(mins)
    end
    return time
end

println(convertTimeToSlotIndex("02:15"))
println(convertSlotIndexToTime(10))

10
02:15


In [2]:
# Using DataFrames to represent tasks

using DataFrames

tasks = DataFrame(
    name = String[], 
    num_hours = Int[], 
    start_day = Int[], 
    start_time_slot = Int[],
    end_day = Int[],
    end_time_slot = Int[],
    is_optional = Bool[],
    is_flexible = Bool[],
    break_after_minutes = Int[],
    min_allotted_hours = Int[],
)

push!(tasks, Dict(
    :name => "Research", 
    :num_hours => 20, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("08:30"), 
    :is_optional => false, 
    :is_flexible => true, 
    :break_after_minutes => 60, 
    :min_allotted_hours => 3
))

push!(tasks, Dict(
    :name => "Edge Computing Course", 
    :num_hours => 10, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("16:00"), 
    :is_optional => false, 
    :is_flexible => true, 
    :break_after_minutes => 60, 
    :min_allotted_hours => 2
))

push!(tasks, Dict(
    :name => "Paper Reading For DB Reading Group", 
    :num_hours => 4, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Tuesday), 
    :end_time_slot => convertTimeToSlotIndex("17:00"), 
    :is_optional => false, 
    :is_flexible => false, 
    :break_after_minutes => 30, 
    :min_allotted_hours => 2
))

push!(tasks, Dict(
    :name => "DB Reading Group", 
    :num_hours => 1, 
    :start_day => Int(Tuesday), 
    :start_time_slot => convertTimeToSlotIndex("17:00"), 
    :end_day => Int(Tuesday), 
    :end_time_slot => convertTimeToSlotIndex("18:00"), 
    :is_optional => false, 
    :is_flexible => false, 
    :break_after_minutes => default, 
    :min_allotted_hours => 1
))

push!(tasks, Dict(
    :name => "Edge Computing - Read PyWren", 
    :num_hours => 2, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("16:00"), 
    :is_optional => false, 
    :is_flexible => false, 
    :break_after_minutes => 30, 
    :min_allotted_hours => 2
))

push!(tasks, Dict(
    :name => "Edge Computing - Read Google Scheduling", 
    :num_hours => 1, 
    :start_day => default, 
    :start_time_slot => default, 
    :end_day => Int(Friday), 
    :end_time_slot => convertTimeToSlotIndex("16:00"), 
    :is_optional => false, 
    :is_flexible => false, 
    :break_after_minutes => default, 
    :min_allotted_hours => 1
))

,name,num_hours,start_day,start_time_slot,end_day,end_time_slot,is_optional,is_flexible,break_after_minutes,min_allotted_hours
,String,Int64,Int64,Int64,Int64,Int64,Bool,Bool,Int64,Int64
1,Research,20,0,0,7,35,false,true,60,3
2,Edge Computing Course,10,0,0,7,65,false,true,60,2
3,Paper Reading For DB Reading Group,4,0,0,4,69,false,false,30,2
4,DB Reading Group,1,4,69,4,73,false,false,0,1
5,Edge Computing - Read PyWren,2,0,0,7,65,false,false,30,2
6,Edge Computing - Read Google Scheduling,1,0,0,7,65,false,false,0,1


In [3]:
# PROBLEM MODEL

using JuMP, Cbc

m = Model(solver=CbcSolver())

# We have a todo 2-d array for each task.
# todo[i][j][k] = 1 means the user should do task i on day j slot id k.
num_tasks, _ = size(tasks)
@variable(m, 0 <= todo[1:(num_tasks), 1:num_days, 1:num_slots_per_day] <= 1, Int)

# We have one more array for breaks/leisure
@variable(m, 0 <= leisure[1:num_days, 1:num_slots_per_day] <= 1, Int)

7×96 Array{Variable,2}:
 leisure[1,1]  leisure[1,2]  leisure[1,3]  …  leisure[1,95]  leisure[1,96]
 leisure[2,1]  leisure[2,2]  leisure[2,3]     leisure[2,95]  leisure[2,96]
 leisure[3,1]  leisure[3,2]  leisure[3,3]     leisure[3,95]  leisure[3,96]
 leisure[4,1]  leisure[4,2]  leisure[4,3]     leisure[4,95]  leisure[4,96]
 leisure[5,1]  leisure[5,2]  leisure[5,3]     leisure[5,95]  leisure[5,96]
 leisure[6,1]  leisure[6,2]  leisure[6,3]  …  leisure[6,95]  leisure[6,96]
 leisure[7,1]  leisure[7,2]  leisure[7,3]     leisure[7,95]  leisure[7,96]

In [4]:
# We can only be doing one type of task every slot
for j=1:num_days
    for k=1:num_slots_per_day
        @constraint(m, sum(todo[i, j, k] for i=1:num_tasks) + leisure[j, k] == 1)
    end
end

In [5]:
# Cannot work outside working hours
for j=1:num_days
    for k=1:(convertTimeToSlotIndex(work_start)-1)
        @constraint(m, leisure[j, k] == 1)
    end
end

for j=1:num_days
    for k=convertTimeToSlotIndex(work_end):num_slots_per_day
        @constraint(m, leisure[j, k] == 1)
    end
end

In [6]:
# Number of time slots spent on each task during the week in the specified start-end time window 
@variable(m, slots_spent[1:num_tasks] >= 0, Int)

function getTimeWindow(tid)
    task = tasks[tid, :]
    start_day = task.start_day
    if start_day == default
        start_day = Int(Saturday)
    end
    start_time_slot = task.start_time_slot
    if start_time_slot == default
        start_time_slot = 1
    end

    end_day = task.end_day
    if end_day == default
        end_day = Int(Friday)
    end
    end_time_slot = task.end_time_slot
    if end_time_slot == default
        end_time_slot = num_slots_per_day
    end
    return start_day, start_time_slot, end_day, end_time_slot
end

for i=1:num_tasks
    start_day, start_time_slot, end_day, end_time_slot = getTimeWindow(i)
    if start_day == end_day
        @constraint(m, slots_spent[i] == sum(todo[i, start_day, k] for k=start_time_slot:end_time_slot-1))
    elseif end_day == start_day + 1
        @constraint(m, slots_spent[i] == sum(todo[i, start_day, k] for k=start_time_slot:num_slots_per_day)  +
                sum(todo[i, end_day, k] for k=1:end_time_slot-1))
    else
        @constraint(m, slots_spent[i] == sum(todo[i, start_day, k] for k=start_time_slot:num_slots_per_day)  +
                sum(todo[i, j, k] for k=1:num_slots_per_day for j=start_day+1:end_day-1) +
                sum(todo[i, end_day, k] for k=1:end_time_slot-1))
    end
end

In [7]:
# Should spend the required amount of time for tasks
for i=1:num_tasks
    task = tasks[i, :]
    if task.is_optional
        @constraint(m, slots_spent[i]*15 <= task.num_hours*60)
    else
        @constraint(m, slots_spent[i]*15 == task.num_hours*60)
    end
end

In [8]:
# Should have a leisure slot after break_after_minutes duration
for i=1:num_tasks
    task = tasks[i, :]
    break_after_minutes = task.break_after_minutes
    if break_after_minutes == default
        continue
    end
    break_after_slots = Int(break_after_minutes/15)
    for j=1:num_days
        for k=1:num_slots_per_day-break_after_slots
            @constraint(m, sum(todo[i, j, l] for l=k:k+break_after_slots) <= break_after_slots)
            @constraint(m, sum(todo[i, j, l] for l=k:k+break_after_slots) +
                sum(leisure[j, l] for l=k:k+break_after_slots) <= break_after_slots + 1)
        end
    end
end

In [9]:
# Should have a leisure slot in between two different tasks
for t1=1:num_tasks-1
    for t2=t1+1:num_tasks
        for j=1:num_days
            for k=1:num_slots_per_day-2
                @constraint(m, todo[t1, j, k] + todo[t2, j, k+2] - leisure[j, k+1] <= 1)
                @constraint(m, todo[t2, j, k] + todo[t1, j, k+2] - leisure[j, k+1] <= 1)
            end
        end
    end
end

In [10]:
# Defining a variable to differentiate break in between tasks, from a constant break such as sleeping. 
@variable(m, 0 <= is_in_between_break[1:num_days, 1:num_slots_per_day] <= 1, Int)

# Cannot have in between breaks outside working hours, as no tasks are scheduled
for j=1:num_days
    for k=1:(convertTimeToSlotIndex(work_start)-1)
        @constraint(m, is_in_between_break[j, k] == 0)
    end
end

for j=1:num_days
    for k=convertTimeToSlotIndex(work_end):num_slots_per_day
        @constraint(m, is_in_between_break[j, k] == 0)
    end
end

# During working hours, if it is a break in between tasks, 
# then the next time slot will have leisure value 0
for j=1:num_days
    for k=convertTimeToSlotIndex(work_start):convertTimeToSlotIndex(work_end)-1
        @constraint(m, is_in_between_break[j, k] >= leisure[j, k] - leisure[j, k+1])
    end
end

In [11]:
# Work window on each day, and hours of work exceeded on each day of the week
@variable(m, work_window[1:num_days] >= 0, Int)
@variable(m, hours_exceeded[1:num_days] >= 0, Int)
for j=1:num_days
    @constraint(m, work_window[j] == sum(todo[i, j, k] for i=1:num_tasks for k=1:num_slots_per_day) +
        sum(is_in_between_break[j, k] for k=1:num_slots_per_day))
    if j==Int(Saturday) || j==Int(Sunday)
        @constraint(m, hours_exceeded[j] >= work_window[j]/4 - preferred_working_hours_weekend)
    else
        @constraint(m, hours_exceeded[j] >= work_window[j]/4 - preferred_working_hours_weekday)
    end
end

In [12]:
# TRIAL for min hours allocated

# Minimum alloted duration of work while scheduling a task
# We first compute the start time of each task
# @variable(m, 0 <= task_started[1:num_tasks, 1:num_days, 1:num_slots_per_day] <= 1, Int)
# for i=1:num_tasks
#     for j=1:num_days
#         for k=3:num_slots_per_day
#             @constraint(m, task_started[i, j, todo= todo[i, j, k]*(1 - todo[i, j, k-1] - todo[i, j, k-2]))
#         end
#     end
# end

# # Given the task start times, we constraint the durationum_tasksork
# for i=1:num_tasks
#     task = tasks[i, :]
#     break_after_minutes = task.break_after_minutes
#     if break_after_minutes != defaultbreak      break_after_slots = Int(break_after_mins/4)
#         num_continuous_slots = task.min_allotted_hours*4
#         num_continuous_slots += div(num_continuous_slots, break_after_slots)
#     else
#         num_continuous_slots = task.min_allotted_hours*4
#     num_slots_per_day   for j=1:num_days
#         for k=1:num_slots_per_day
            
#         end
#     end
# end

In [13]:
# TRIAL for task switches - The solver did not finish running
# Number of task switches in the schedule
# @variable(m, 0 <= task_switches[1:num_tasks, 1:num_days, 1:num_slots_per_day] <= 1, Int)
# for i=1:num_tasks
#     for j=1:num_days
#         for k=1:num_slots_per_day-2
#             @constraint(m, task_switches[i, j, k] >= todo[i, j, k] - todo[i, j, k+2])
#         end
#     end
# end

In [14]:
@objective(m, Min, sum(hours_exceeded[j] for j=1:num_days) +   # Number of hours exceeded over preferred hours
#     sum(task_switches[i, j, k] for k=1:num_slots_per_day for j=1:num_days for i=1:num_tasks) +  # Num task switches
    sum(tasks[i, :].num_hours - slots_spent[i]/4 for i=1:num_tasks))  # Number of unfulfilled tasks
status = solve(m)

:Optimal

In [15]:
todo = getvalue(todo)
for i=1:num_tasks
    task = tasks[i, :]
    println(task.name, " schedule:")
    for j=1:num_days
        println(day_names[j], ": ", sum(todo[i, j, k] for k=1:num_slots_per_day)/4, " hours")
    end
    println()
end

Research schedule:
Saturday: 1.0 hours
Sunday: 1.75 hours
Monday: 2.5 hours
Tuesday: 4.25 hours
Wednesday: 6.25 hours
Thursday: 4.25 hours
Friday: 2.75 hours

Edge Computing Course schedule:
Saturday: 0.0 hours
Sunday: 0.5 hours
Monday: 3.25 hours
Tuesday: 0.5 hours
Wednesday: 1.25 hours
Thursday: 1.0 hours
Friday: 4.0 hours

Paper Reading For DB Reading Group schedule:
Saturday: 0.75 hours
Sunday: 0.25 hours
Monday: 1.75 hours
Tuesday: 1.25 hours
Wednesday: 0.0 hours
Thursday: 0.0 hours
Friday: 0.0 hours

DB Reading Group schedule:
Saturday: 0.0 hours
Sunday: 0.0 hours
Monday: 0.0 hours
Tuesday: 1.0 hours
Wednesday: 0.0 hours
Thursday: 0.0 hours
Friday: 0.0 hours

Edge Computing - Read PyWren schedule:
Saturday: 0.0 hours
Sunday: 0.0 hours
Monday: 0.0 hours
Tuesday: 0.0 hours
Wednesday: 0.0 hours
Thursday: 1.5 hours
Friday: 0.5 hours

Edge Computing - Read Google Scheduling schedule:
Saturday: 0.25 hours
Sunday: 0.0 hours
Monday: 0.0 hours
Tuesday: 0.25 hours
Wednesday: 0.0 hours
Thur